In [3]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    name: str

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [5]:
from langchain_core.tools import tool

@tool(return_direct=True)
def call_api_get(endpoint: str, headers: dict) -> str:
    """
    Make API call with GET method.

    Args:
        endpoint (str)
        headers (dict)

    Returns:
        str: api response
    """
    print("GET", endpoint)
    response = {}
    response["status_code"] = 200
    response["body"] = """
    {
        "status": "UP",
        "error": null,
        "version": "development"
    }
    """
    return str(response)

@tool(return_direct=True)
def call_api_post(endpoint: str, body: str, headers: dict) -> str:
    """
    Make API call with POST method.

    Args:
        endpoint (str)
        body (str)
        headers (dict)

    Returns:
        str: api response
    """
    print("POST", endpoint)
    response = {}
    response["status_code"] = 500
    response["body"] = """
    error
    """
    return str(response)

In [11]:
test_cases = [
    """
    Test case: Health check success
    Endpoint: [GET] /api/v1/actuator/health
    Expected response
    - http status code 200
    - response body contains
        - "status": "UP"
        - "version": "development"
    """,
    """
    Test case: Dependency check success
    Endpoint: [POST] /api/v1/dependency-check
    Input:
        body:```{
            "dependency": "kafka"
        }```
    Expected response
    - http status code 200
    - response header contains
        - "Content-Type": "application/json"
    - response body contains
        - "status": "UP"
        - "version": "development"
    """
]

In [12]:
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner, planners
from langchain.agents.tools import Tool

planner_prompt = """
You are a developer and you need to test an API endpoint.
You will be provide api specification, test case and expected response.
Let's first understand provided input and create a plan to test the API.
Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps.
Please make the plan the minimum number of steps required to accurately complete the test.
The final step should be 'Report summary result of all previous steps if any failed or all passed.'
At the end of your plan, say '<END_OF_PLAN>'
No preamble or explanation is needed, just the plan.
"""


agent_tools = [call_api_get,
               call_api_post]

planner = load_chat_planner(llm, system_prompt=planner_prompt)
result = planner.llm_chain.invoke(test_cases[1])
print(result["text"])
# executor = load_agent_executor(llm, agent_tools, verbose=True)

# import uuid
# thread_id = str(uuid.uuid4())

# metadata = {
#     "thread_id": thread_id
# }
# agent = PlanAndExecute(planner=planner, executor=executor, verbose=True, metadata=metadata)

# agent.run(test_cases[0])

Plan:
1. Prepare the request body with the JSON payload: `{"dependency": "kafka"}`.
2. Send a POST request to the endpoint `/api/v1/dependency-check` with the prepared request body.
3. Verify the HTTP status code of the response is 200.
4. Verify the response header contains "Content-Type" with the value "application/json".
5. Verify the response body contains the key "status" with the value "UP".
6. Verify the response body contains the key "version" with the value "development".
7. Report summary result of all previous steps if any failed or all passed.
<END_OF_PLAN>
